In [6]:
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pylab as plt
from torch.utils.data import Dataset, DataLoader
import copy
from tqdm import tqdm
import torch.optim.lr_scheduler as sched
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score, roc_curve
import pickle
import os

In [7]:
root = os.getcwd()
data_root = os.path.join(root, 'data')

### Prepare Data

In [8]:
train_X = pickle.load(open(os.path.join(data_root, 'train_X.pickle'), "rb"))
train_y = pickle.load(open(os.path.join(data_root, 'train_y.pickle'), "rb"))
test_X = pickle.load(open(os.path.join(data_root, 'test_X.pickle'), "rb"))
test_y = pickle.load(open(os.path.join(data_root, 'test_y.pickle'), "rb"))

In [194]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class ICUDataset(Dataset):
    
    def __init__(self, input_array, label_df):
        # Args:
        #      input_array: list of list
        #      label_array: list of list
        
        super(ICUDataset, self).__init__()
        self.input_tensors = []
        self.labels = torch.tensor(list(label_df.iloc[:,-1]), dtype=torch.long)
        for sample in input_array:
            self.input_tensors.append(torch.tensor([sample[:]], dtype=torch.float))
        
    def __len__(self):
        return len(self.input_tensors)

    def __getitem__(self, index):
        x = self.input_tensors[index]
        # want to have y as tensor([1]), for 1 being class
        y = self.labels[index]
                   
        return x, y

In [195]:
data_train = ICUDataset(train_X, train_y)
data_test = ICUDataset(test_X, test_y)

In [196]:
# Test data train
x, y = next(iter(data_train))
x.shape
y.shape

torch.Size([])

In [197]:
data_train_loader = DataLoader(data_train, batch_size = 64, shuffle=False, num_workers=4)
data_test_loader = DataLoader(data_test, batch_size = 64, shuffle=False, num_workers=4)

In [198]:
# We want to have x.shape = [batch_size, height, width = len of input]
# y.shape = [batch_size, 1] because y is a class
x, y = next(iter(data_train_loader))

In [199]:
y.sha

torch.Size([64])

### Model

In [260]:
class MLPNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        x = x.contiguous().view(-1,input_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    


### Train and Valid

In [304]:
def train(model, data_train_loader, data_train, optimizer, criterion):
    y_true_list_train = []
    y_score_list_train = []
    num_correct = 0
    cur_loss = 0
    losslisttrain = []
    
    model.train()
    
    for iter_, (inputs, targets) in enumerate(data_train_loader):
#         if iter_ % 500 == 0:
#             print("Train Phase: Iteration {}/{}".format(iter_+1, len(data_train_loader)))

        # zero out the parameter gradients
        optimizer.zero_grad()

        # Setup for forward
        inputs = inputs.to(device)
        targets = targets.to(device)
        batch_size = inputs.size(0)


        # Feed forward to get the logits
        logit = model(inputs)
        loss = criterion(logit, targets)
        loss_val = loss.item()
        cur_loss += loss_val * batch_size
        #print('cur_loss', cur_loss)

        # Add y_true and y_pred to list to calculate AUC score
        y_true_list_train += targets.data.tolist()
        y_pred = logit.argmax(dim=1)
        num_correct += (y_pred == targets).sum().item()
        y_score_list_train += logit.data.numpy()[:, 1].tolist()

        # Backward
        loss.backward()
        optimizer.step()


    avg_train_loss = cur_loss / len(data_train)
    losslisttrain.append(avg_train_loss)

    train_auc =  roc_auc_score(y_true=(np.array(y_true_list_train)==1),
                               y_score=np.array(y_score_list_train))


    train_acc = (num_correct / len(data_train)) * 100 
    print(f'Train Loss: {avg_train_loss:.4f}')
    print(f'Train AUC: {train_auc:.4f}')
    print(f'Train ACC: {train_acc:.4f}')

    return(avg_train_loss, train_auc, train_acc, model)

In [305]:
def test(model, data_test_loader, data_test):
    y_true_list_test = []
    y_score_list_test = []
    num_correct = 0
    cur_loss = 0
    losslisttest = []
    
    model.eval()
    
    for iter_, (inputs, targets) in enumerate(data_test_loader):
#         if iter_ % 500 == 0:
#             print("Valid Phase: Iteration {}/{}".format(iter_+1, len(data_test_loader)))
        # Setup for forward
        inputs = inputs.to(device)
        targets = targets.to(device)
        batch_size = inputs.size(0)

        # Feed forward to get the logits
        logit = model(inputs)
        loss = criterion(logit, targets)
        loss_val = loss.item()
        cur_loss += loss_val * batch_size
        #print('cur_loss', cur_loss)

        # Add y_true and y_pred to list to calculate AUC score
        y_true_list_test += targets.data.tolist()
        y_pred = logit.argmax(dim=1)
        num_correct += (y_pred == targets).sum().item()
        y_score_list_test += logit.data.numpy()[:, 1].tolist()



    avg_test_loss = cur_loss / len(data_test)
    losslisttest.append(avg_test_loss)

    test_auc =  roc_auc_score(y_true=(np.array(y_true_list_test)==1),
                               y_score=np.array(y_score_list_test))


    test_acc = (num_correct / len(data_test)) * 100 
    print(f'Valid Loss: {avg_test_loss:.4f}')
    print(f'Valid AUC: {test_auc:.4f}')
    print(f'Valid ACC: {test_acc:.4f}')

    return(avg_test_loss, test_auc, test_acc)

In [314]:
def train_valid(model, data_train_loader, data_train, optimizer, criterion,
                data_test_loader, data_test, num_epochs):
    best_val_loss = 1000
    best_weights = copy.deepcopy(model.state_dict())
    history_train = {'loss': [], 'acc': [], 'auc': []}
    history_test = {'loss': [], 'acc': [], 'auc': []}

    for epoch in range(num_epochs):
        avg_train_loss, train_auc, train_acc, model = train(model, data_train_loader,
                                                            data_train, optimizer, criterion)
        avg_test_loss, test_auc, test_acc = test(model, data_test_loader, data_test)
        history_train['loss'].append(avg_train_loss)
        history_train['auc'].append(train_auc)
        history_train['acc'].append(train_acc)
        history_test['loss'].append(avg_test_loss)
        history_test['auc'].append(test_auc)
        history_test['acc'].append(test_acc)
        
        
        
        if avg_test_loss < best_val_loss:
            best_val_loss = avg_test_loss
            best_weights = copy.deepcopy(model.state_dict())
            
        print('-'*20)
    return (best_weights, metrics_dict)

### Training and Validating


#### 1. MLP Model

In [315]:
# nn package also has different loss functions.
# we use Cross Entropy loss for our regression task
criterion = torch.nn.CrossEntropyLoss()

# we use the optim package to apply
# stochastic gradient descent for our parameter updates
# built-in L2
learning_rate = 1e-3
lambda_l2 = 1e-5

input_size = 412
hidden_size = 200
output_size = 2

model = MLPNet(input_size, hidden_size, output_size)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) 
num_epochs = 10

In [316]:
best_weights, metrics_dict = train_valid(model, data_train_loader, data_train,
                                         optimizer, criterion,
                                         data_test_loader, data_test, num_epochs)

Train Loss: 0.3419
Train AUC: 0.6225
Train ACC: 91.2812
Valid Loss: 0.2856
Valid AUC: 0.7438
Valid ACC: 90.9230
--------------------
Train Loss: 0.2562
Train AUC: 0.7819
Train ACC: 91.7010
Valid Loss: 0.2523
Valid AUC: 0.8076
Valid ACC: 91.3864
--------------------
Train Loss: 0.2341
Train AUC: 0.8223
Train ACC: 92.1112
Valid Loss: 0.2374
Valid AUC: 0.8307
Valid ACC: 91.7407
--------------------
Train Loss: 0.2232
Train AUC: 0.8402
Train ACC: 92.2707
Valid Loss: 0.2293
Valid AUC: 0.8428
Valid ACC: 91.8498
--------------------
Train Loss: 0.2169
Train AUC: 0.8501
Train ACC: 92.3947
Valid Loss: 0.2241
Valid AUC: 0.8504
Valid ACC: 92.0079
--------------------
Train Loss: 0.2125
Train AUC: 0.8568
Train ACC: 92.4751
Valid Loss: 0.2203
Valid AUC: 0.8559
Valid ACC: 92.1223
--------------------
Train Loss: 0.2093
Train AUC: 0.8619
Train ACC: 92.5419
Valid Loss: 0.2174
Valid AUC: 0.8603
Valid ACC: 92.2259
--------------------
Train Loss: 0.2066
Train AUC: 0.8659
Train ACC: 92.5773
Valid Loss: 0

In [ ]:
    # Generate diagnostic plots for the loss and accuracy
fig, axes = plt.subplots(ncols=2, figsize=(9, 4.5))
for ax, metric in zip(axes, ['loss', 'acc']):
    ax.plot(history_train[metric])
    ax.plot(history_test[metric])
    ax.set_xlabel('epoch', fontsize=12)
    ax.set_ylabel(metric, fontsize=12)
    ax.legend(['Train', 'Test'], loc='best')
plt.show()


In [313]:
plt(metrics_dict['train_loss']

{'train_loss': [0.358191200780888,
  0.2600617195867105,
  0.2371008291074881,
  0.2259151222011415,
  0.21914252340915466,
  0.2144179266708268,
  0.21084155164688473,
  0.2079680905138031,
  0.205564409329486,
  0.20351347075799384],
 'train_auc': [0.5260674989520792,
  0.7357140153954882,
  0.79437409512393,
  0.8203196285249976,
  0.8346378306924611,
  0.8441870736442694,
  0.8512599232288889,
  0.856805858118028,
  0.8612875827349689,
  0.8649775738944705],
 'train_acc': [90.26850211258008,
  91.66825678070056,
  92.04988414883466,
  92.22570532915361,
  92.33610467493526,
  92.42469674253783,
  92.47921493798556,
  92.51465176502658,
  92.61278451683249,
  92.65912498296306],
 'test_loss': [0.29151189954383666,
  0.256300459144056,
  0.2409850781635104,
  0.23244021789319108,
  0.2267496171849885,
  0.2225706548949582,
  0.21932385395983864,
  0.2166872709097574,
  0.2144796348726815,
  0.212603324746927],
 'test_auc': [0.676206558246065,
  0.7712491787188244,
  0.806584621825863

In [ ]:
def train_valid(model, criterion, optimizer, epoch_num):

losslisttrain = []
losslisttest = []
best_val_loss = 100

for epoch in range(100):
    print('Epoch {}'.format(epoch))
    
    y_true_list_train = []
    y_score_list_train = [] # score of the malignant class - class 2
    y_true_list_test = []
    y_score_list_test = [] # score of the malignant class - class 2
    
    best_weights = copy.deepcopy(model.state_dict())
    model.train()

    cur_loss = 0
#     with torch.enable_grad(), \
#         tqdm(total=len(data_train_loader)) as progress_bar:
    for iter_, (x, y_true) in enumerate(data_train_loader):

        if iter_ % 10:
            print(f'Train Phase:  Iteration {iter_+1}/{len(data_train_loader)}', end="\r")

        # zero out the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):

            # Setup for forward
            x = x.to(device)
            y_true = y_true.to(device)
            batch_size = x.size(0)
            optimizer.zero_grad()

            # Feed forward to get the logits
            logit = model(x)
            loss = criterion(logit, y_true)
            loss_val = loss.item()
            cur_loss += loss_val * batch_size
            #print('cur_loss', cur_loss)

            # Add y_true and y_pred to list to calculate AUC score
            y_true_list_train += y_true.data.tolist()
            y_pred = logit.data.numpy().ravel()[2]
            y_score_list_train += logit.data.numpy()[:, 2].tolist()

        # Backward
        loss.backward()
        optimizer.step()

#             progress_bar.update(batch_size)
#             progress_bar.set_postfix(epoch=epoch,
#                          NLL=loss_val)


    avg_train_loss = cur_loss / len(data_train)
    losslisttrain.append(avg_train_loss)
    # get the auc
    train_auc =  roc_auc_score(y_true=(np.array(y_true_list)==2), y_score=np.array(y_score_list))


    print(f'Train Loss: {avg_train_loss:.4f}')
    print(f'Train AUC: {train_auc:.4f}')


    ######### Evaluation #####

    val_loss = 0

    for iter_, (x, y_true) in enumerate(data_test_loader):
        if iter_ % 10:
            print(f'Test Phase:  Iteration {iter_+1}/{len(data_test_loader)}', end="\r")
        # Setup for forward
        x = x.to(device)
        y_true = y_true.to(device)
        batch_size = x.size(0)

        # Feed forward to get the logits
        logit = model(x)
        loss = criterion(logit, y_true)
        loss_val = loss.item()
        val_loss += loss_val * batch_size
        #print('cur_loss', cur_loss)

        # Add y_true and y_pred to list to calculate AUC score
        y_true_list_test += y_true.data.tolist()
        y_pred = logit.data.numpy().ravel()[2]
        y_score_list_test += logit.data.numpy()[:, 2].tolist()

    avg_val_loss = cur_loss / len(data_test)
    losslisttest.append(avg_val_loss)
    # get the auc
    val_auc =  roc_auc_score(y_true=(np.array(y_true_list_test)==2),
                             y_score=np.array(y_score_list_test))


    print(f'Val Loss: {avg_val_loss:.4f}')
    print(f'Val AUC: {val_auc:.4f}')


    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_weights = copy.deepcopy(model.state_dict())

#             torch.save({
#             'state_dict': EncoderTransformer_model.state_dict()
#                 }, f'./chat_EncoderTransformer_model_best_{epoch}.pt')